# Guardian - Iteration 9

## Import Packages

In [1]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [2]:
iteration = 9

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2018-09-01",
    "to-date": "2019-02-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY4"),
}


In [3]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [4]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [5]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2018-09-01&to-date=2019-02-01&order-by=newest&api-key=295c6540-8c34-4390-ad47-0cf94ebc153f&page=1
status ok
total 33045
startIndex 1
pageSize 50
pages 661
orderBy newest
---- RUNTIME STATUS ---- 
(1/661) in 0:00:17.692015s
(2/661) in 0:00:05.170146s
(3/661) in 0:00:09.493010s
(4/661) in 0:00:11.572534s
(5/661) in 0:00:10.253765s
(6/661) in 0:00:04.175537s
(7/661) in 0:00:08.198564s
(8/661) in 0:00:05.150103s
(9/661) in 0:00:04.361106s
(10/661) in 0:00:07.797253s
(11/661) in 0:00:10.465306s
(12/661) in 0:00:05.731905s
(13/661) in 0:00:06.848337s
(14/661) in 0:00:09.237614s
(15/661) in 0:00:09.820265s
(16/661) in 0:00:09.832713s
(17/661) in 0:00:07.323508s
(18/661) in 0:00:13.756364s
(19/661) in 0:00:15.881095s
(20/661) in 0:00:40.722537s
(21/661) in 0:00:13.883717s
(22/661) in 0:00:11.177591s
(23/661) in 0:00:08.418213s
(24/661) in 0:00:11.453728s
(25/661) i

In [6]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}.csv.zip", compression="zip")
    return df

In [7]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(33045, 13) took 0:00:00.009604s

unfold_columns:
 shape=(33045, 49) took 0:00:07.142100s

drop_columns:
 shape=(33045, 31) took 0:00:00.018423s

create_csv:
 shape=(33045, 31) took 0:00:23.299142s

